# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846855428342                   -0.70    4.2         
  2   -7.852295019513       -2.26       -1.53    1.0   17.4ms
  3   -7.852614284290       -3.50       -2.56    1.8   22.5ms
  4   -7.852645999064       -4.50       -2.90    2.5   34.8ms
  5   -7.852646522140       -6.28       -3.21    1.0   19.4ms
  6   -7.852646679691       -6.80       -4.11    1.0   17.6ms
  7   -7.852646686570       -8.16       -5.29    1.8   21.2ms
  8   -7.852646686724       -9.81       -5.42    1.8   20.9ms
  9   -7.852646686728      -11.41       -5.64    1.2   18.4ms
 10   -7.852646686730      -11.68       -6.58    1.0   18.2ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846871182472                   -0.70           4.8         
  2   -7.852553401090       -2.25       -1.63   0.80    2.0    222ms
  3   -7.852639206153       -4.07       -2.73   0.80    1.0   16.4ms
  4   -7.852646582390       -5.13       -3.36   0.80    2.0   20.2ms
  5   -7.852646679166       -7.01       -4.32   0.80    1.2   16.8ms
  6   -7.852646686595       -8.13       -4.82   0.80    2.0   20.4ms
  7   -7.852646686719       -9.91       -5.55   0.80    1.2   17.2ms
  8   -7.852646686730      -10.97       -6.41   0.80    2.2   21.2ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.369147e+01     3.396804e+00
 * time: 0.04660916328430176
     1     1.158552e+00     1.926141e+00
 * time: 0.22212600708007812
     2    -1.510943e+00     2.308656e+00
 * time: 0.2401731014251709
     3    -3.809331e+00     1.950597e+00
 * time: 0.2659890651702881
     4    -4.959102e+00     1.843419e+00
 * time: 0.2917470932006836
     5    -6.754448e+00     1.054595e+00
 * time: 0.3176090717315674
     6    -7.402633e+00     6.594833e-01
 * time: 0.3432772159576416
     7    -7.672100e+00     5.093120e-01
 * time: 0.36113715171813965
     8    -7.760325e+00     1.641525e-01
 * time: 0.3789820671081543
     9    -7.811002e+00     8.495859e-02
 * time: 0.3969550132751465
    10    -7.834883e+00     8.032705e-02
 * time: 0.4744691848754883
    11    -7.841936e+00     6.747633e-02
 * time: 0.49250316619873047
    12    -7.845915e+00     3.531924e-02
 * time: 0.5103771686553955
    13    -7.850068e+00     2.610939e-02
 * time: 0.528205

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846906936640                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645906804                   -1.65         
  2   -7.852646686730       -6.11       -3.72    1.50s
  3   -7.852646686730      -13.28       -7.23    119ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.001457291571667e-7
|ρ_newton - ρ_scfv| = 3.476973564724203e-8
|ρ_newton - ρ_dm|   = 4.559957502746101e-10
